**대회 목표:** 

대회의 데이터셋은 자유 이용 저작물인 으스스한 작가들이 쓴 소설 작품 텍스트입니다:  

1. Edgar Allan Poe (EAP)
2. HP Lovecraft (HPL)
3. Mary Wollstonecraft Shelley (MWS)

목표은 테스트 셋의 문장들의 작가를 정확하게 구분하는 것입니다.

**노트북의 목표:**

이 노트북에서는 spooky 작가들을 구분하는 데에 도움을 주는 다양한 feature들을 만들기를 시도할 것입니다.  

첫번째 단계로, 우리는 피처 엔지니어링 파트를 깊게 공부하기 전에 기본적인 데이터 시각화와 전처리를 할 것입니다.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import string
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble, metrics, model_selection, naive_bayes
color = sns.color_palette()

%matplotlib inline

eng_stopwords = set(stopwords.words('english'))
pd.options.mode.chained_assignment = None

In [ ]:
train_df = pd.read_csv('../input/spooky-author-identification/train.zip')
test_df = pd.read_csv('../input/spooky-author-identification/test.zip')
print(f"Number of rows in train dataset : {train_df.shape[0]}")
print(f"Number of rows in test dataset : {test_df.shape[0]}")

In [ ]:
train_df.head()

클래스가 균형있게 분포했는지를 확인하고자 각 작가들의 출현빈도를 확인해보겠습니다.

In [ ]:
cnt_srs = train_df['author'].value_counts()

plt.figure(figsize=(8,4))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8)
plt.ylabel('Number of Occurences', fontsize=12)
plt.xlabel('Author Name', fontsize=12)
plt.show()

아주 좋네요. 클래스 불균형이 있는 것처럼 보이지는 않습니다. 이제 가능하다면 각 작가들의 문체를 확인해봅시다.

In [ ]:
grouped_df = train_df.groupby('author')

for name, group in grouped_df:
    print(f"Author name : {name}")
    cnt = 0
    
    for idx, row in group.iterrows():
        print(row['text'])
        cnt += 1
        
        if cnt == 5:
            break
    
    print("\n")

테스트 데이터를 보면 몇개의 꽤 특별한 문자들을 볼 수 있습니다. 그래서 이 특별한 문자들의 횟수는 좋은 feature가 될 수 있습니다. 나중에 이 문자들을 처리하도록 하겠습니다.

그 외에는 따로 단서가 있지는 않습니다... 만약에 흥미로운 스타일(만들 수 있는 feature)이 있다면 댓글로 달아주세요!

**Feature Engineering (특징 추출):**

이제 feature engineerin을 해보도록 하겠습니다. 이 과정은 2개의 주요 파트로 구성됩니다:

1. Meta fetures - 단어의 수, 불용어의 수, 구두점의 수와 같은 텍스트로부터 뽑아낼 수 있는 특징들입니다.
2. Text based features - 빈도수, svd, word2vec등과 같은 텍스트/단어를 직접적으로 기반으로 하는 특징들입니다.

**Meta Features:**

우리는 meata 특징들을 만들고 얼마나 그것들이 작가들을 잘 예측하는지 보겠습니다.  
feature의 종류는 아래와 같습니다:

1. 텍스트에 있는 모든 단어의 수
2. 텍스트에 있는 중복을 제외한 모든 단어의 수
3. 텍스트에 있는 문장의 길이
4. 불용어의 수
5. 구두점의 수
6. 대문자 단어들의 수
7. 제목의 역할을 하는 단어의 수
8. 단어들의 평균길이

In [ ]:
## Number of words in the text ##
train_df['num_words'] = train_df['text'].apply(lambda x: len(str(x).split()))
test_df['num_words'] = test_df['text'].apply(lambda x: len(str(x).split()))

## Number of unique words in the text ##
train_df['num_unique_words'] = train_df['text'].apply(lambda x: len(set(str(x).split())))
test_df['num_unique_words'] = test_df['text'].apply(lambda x: len(set(str(x).split())))

## Number of characters in the text ##
train_df['num_chars'] = train_df['text'].apply(lambda x: len(str(x)))
test_df['num_chars'] = test_df['text'].apply(lambda x: len(str(x)))

## Number of soptwords in the text ##
train_df['num_stopwrods'] = train_df['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
test_df['num_stopwrods'] = test_df['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))

## Number of punctuations in the text ##
train_df['num_punctuations'] = train_df['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
test_df['num_punctuations'] = test_df['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

## Number of title case words in the text ##
train_df['num_words_upper'] = train_df['text'].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test_df['num_words_upper'] = test_df['text'].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

## Average length of the words in the text ##
train_df['mean_word_len'] = train_df['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test_df['mean_word_len'] = test_df['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

이제 새로 만든 변수들이 예측을 할때 도움이 되는지 확인하기 위해 그래프를 그려봅시다

In [ ]:
train_df['num_words'].loc[train_df['num_words']>80] = 80 # truncation for better visuals
plt.figure(figsize=(12, 8))
sns.violinplot(x='author', y='num_words', data=train_df)
plt.xlabel('Author Name', fontsize=12)
plt.ylabel('Number of words in text', fontsize=12)
plt.title('Number of words by author', fontsize=15)
plt.show()

EAP가 MWS와 HPL보다 모든 단어의 수가 약간 적은 것을 볼 수 있습니다.

In [ ]:
train_df['num_punctuations'].loc[train_df['num_punctuations']>10] = 10 # truncation for better visuals
plt.figure(figsize=(12, 8))
sns.violinplot(x='author', y='num_punctuations', data=train_df)
plt.xlabel('Author Name', fontsize=12)
plt.ylabel('Number of punctuations in text', fontsize=12)
plt.title('Number of punctuations by author', fontsize=15)
plt.show()

이건 다소 쓸모가 있어보입니다. 이제 feature에 기반한 몇 텍스트를 만드는데 집중해봅시다.

우선 이 meta 특징들이 얼마나 도움이 되는지 확인하고자 기본적인 모델을 만들어봅시다.

In [ ]:
## Prepare the data for modeling ##
author_mapping_dict = {'EAP':0, 'HPL':1, 'MWS':2}
train_y = train_df['author'].map(author_mapping_dict)
train_id = train_df['id'].values
test_id = test_df['id'].values

### recompute the truncated variables again ###
train_df['num_words'] = train_df['text'].apply(lambda x: len(str(x).split()))
test_df['num_words'] = test_df['text'].apply(lambda x: len(str(x).split()))
train_df['mean_word_len'] = train_df['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test_df['mean_word_len'] = test_df['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

cols_to_drop = ['id', 'text']
train_X = train_df.drop(cols_to_drop+['author'], axis=1)
test_X = test_df.drop(cols_to_drop, axis=1)

단순히 이 meta 특징들만 사용해서 간단한 XGBoost model을 훈련하겠습니다.

In [ ]:
def runXGB(train_X, train_y, test_X, test_y=None, test_X2=None, seed_val=0, child=1, colsample=0.3):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 3
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = 'mlogloss'
    param['min_child_weight'] = child
    param['subsample'] = 0.8
    param['colsample_bytree'] = colsample
    param['seed'] = seed_val
    num_rounds = 2000
    
    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)
    
    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [(xgtrain, 'train'), (xgtest, 'test')]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=50, verbose_eval=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)
        
    pred_test_y = model.predict(xgtest, ntree_limit=model.best_ntree_limit)
    
    if test_X2 is not None:
        xgtest2 = xgb.DMatrix(test_X2)
        pred_test_y2 = model.predict(xgtest2, ntree_limit=model.best_ntree_limit)
    
    return pred_test_y, pred_test_y2, model

커널의 런타임을 위해 우리는 k-fold 교차검증 점수를 첫번째 fold만 체크할 수 있습니다. 로컬 환경에서 돌린다면 `break`를 제거하고 돌려주세요

In [ ]:
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
cv_scores = []
pred_full_test = 0
pred_train = np.zeros([train_df.shape[0], 3])

for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_X.loc[dev_index], train_X.loc[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runXGB(dev_X, dev_y, val_X, val_y, test_X, seed_val=0)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index, :] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    break

print(f'cv scores : ', cv_scores)

meta feature들을 활용하여 '1.0010'의 mlogloss를 얻었습니다. 나쁘지 않은 점수입니다. 이제 이중 어떤 feature가 중요한지 알아봅시다.

In [ ]:
### Plot the important variables ###
fig, ax = plt.subplots(figsize=(12, 12))
xgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
plt.show()

문장의 길이, 단어의 평균 길이 그리고 중복되지 않은 단어의 개수는 top3의 변수로 확인됩니다. 이제 텍스트를 기반으로하는 feature들을 만들고 확인해봅시다.

**Text Based Features:**

우리가 만들 수 있는 기본적인 feature는 텍스트에 있는 단어의 tf-idf 변수입니다. 이걸로 한번 시작해봅시다.

In [ ]:
### Fit transform the tfidf vectorizer ###
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1, 3))
full_tfidf = tfidf_vec.fit_transform(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['text'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['text'].values.tolist())

이제 우리는 tfidf 벡터를 얻었습니다만 여기엔 까다로운 부분이 있습니다. tfidf의 결과값은 희소 행렬이어서 만약 다른 밀집 특징(feature)들과 사용한다면 2가지 선택지가 존재합니다.

1. tfidf vectorizerfhqnxj 상위 'n'개의 feature들(시스템 구성에 따라 다름)을 선택하고 밀집 형태로 변환하여 다른 feature들과 병합할 수 있습니다.
2. 모델을 단지 희소 특징들을 사용하면 만들고 다른 밀집 특징들 중 하나를 예측에 사용합니다.

데이터셋에 따르면 둘 중 한개는 좋은 성능을 보일 것입니다. 우리는 tfidf의 모든 특징을 사용하는 아주 좋은 채점방식이 있으므로 2번째 접근법을 사용하겠습니다.

또한 Naive Bayes가 이 데이터셋에서 더 좋은 성능을 보이는 것 같습니다. 그래서 우리는 학습이 빠른 tfidf 특징을 사용하는 naive bayes 모델을 설계할 수도 있습니다.

In [ ]:
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    
    return pred_test_y, pred_test_y2, model

**Word Tfidf Vectorizer를 사용한 Naive Bayes**

In [ ]:
cv_scores = []
pred_full_test = 0
pred_train = np.zeros([train_df.shape[0], 3])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)

for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index, :] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))

print(f'Mean cv score : {np.mean(cv_scores)}')
pred_full_test = pred_full_test / 5.

단지 tfidf vectorizer만 사용했는데도 0.842의 mlogloss를 얻었습니다. meta features보다 더 좋습니다. 이제 [혼동 행렬](https://ko.wikipedia.org/wiki/혼동_행렬)을 보겠습니다.

In [ ]:
### Function to create confusion matrix ###
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applies by setting 'normalize=True'
    """
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        # print("Normalized confusionn matrix")
    #else:
    #   print('Confusion matrix, without normalization')
    
    #print(cm)
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                horizontalalignment='center',
                color='white' if cm[i, j] > thresh else 'black')
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cnf_matrix = confusion_matrix(val_y, np.argmax(pred_val_y, axis=1))
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(8, 8))
plot_confusion_matrix(cnf_matrix, classes=['EAP', 'HPL', 'MWS'],
                     title='Confusion matrix, without normalization')
plt.show()

많은 인스턴스가 EAP라고 예측을 했고 해당 클래스에 크게 치우쳐져 있습니다.

**word TFIDF를 사용한 SVD:**

tfidf 벡터가 희소하기 때문에 정보를 압축하고 훨씬 간결하게 표현하는 또 다른 방법은 SVD입니다. 또한 일반적으로 SVD feature들은 과거 텍스트 기반의 대회에서 저에게 좋은 성능을 보여줬습니다. 그래서 우리는 단어 tfidf의 svd feature들을 만들고 우리가 만든 feature셋에 추가하겠습니다.

In [ ]:
n_comp = 20
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(full_tfidf)

train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))

train_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
test_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
train_df = pd.concat([train_df, train_svd], axis=1)
test_df = pd.concat([test_df, test_svd], axis=1)

del full_tfidf, train_tfidf, test_tfidf, train_svd, test_svd

**Word Count Vectorizer을 활용한 Naive Bayes:**

In [ ]:
### Fit transform the count vectorizer ###
tfidf_vec = CountVectorizer(stop_words='english', ngram_range=(1,3))
tfidf_vec.fit(train_df['text'].values.tolist()+test_df['text'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['text'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['text'].values.tolist())

이제 feature들에 기반한 count vectorizer를 사용한 다항 나이브 베이즈 모델을 만들어봅시다.

In [ ]:
cv_scores = []
pred_full_test = 0
pred_train = np.zeros([train_df.shape[0], 3])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index, :] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    
print(f'Mean cv score : {np.mean(cv_scores)}')
pred_full_test = pred_full_test / 5.

# add the predictions as new fetures #
train_df['nb_cvec_eap'] = pred_train[:,0]
train_df['nb_cvec_hpl'] = pred_train[:,1]
train_df['nb_cvec_mws'] = pred_train[:,2]
test_df['nb_cvec_eap'] = pred_full_test[:,0]
test_df['nb_cvec_hpl'] = pred_full_test[:,1]
test_df['nb_cvec_mws'] = pred_full_test[:,2]

In [ ]:
cnf_matrix = confusion_matrix(val_y, np.argmax(pred_val_y, axis=1))
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(8, 8))
plot_confusion_matrix(cnf_matrix, classes=['EAP', 'HPL', 'MWS'],
                     title='Confusion matrix of NB on word count, without normalization')
plt.show()

좋습니다! tfidf vectorizer대신 count vectorizer을 사용한 교차 검증이 mlogloss가 0.451이 나왔습니다.
모델을 사용한 LB 점수는 0.468입니다. 또한 혼동 행렬을 보면 이전보다 훨씬 좋은 것을 알 수 있습니다.

**Character Count Vectorizer를 사용한 Naive Bayes:**

"data eyeballin"의 아이디어는 특별한 문자를 세는 것이 도움을 줄 것입니다. 특별한 문자를 세는 것 대신에 우리는 문자 수준에서 count vectorizer를 사용하여 일부 feature들을 얻을 수 있습니다. 다시 위에서 쓴 다항 나이브 베이즈를 실행해 보겠습니다.

In [ ]:
### Fit transform the tfidf vectorizer ###
tfidf_vec = CountVectorizer(ngram_range=(1,7), analyzer='char')
tfidf_vec.fit(train_df['text'].values.tolist()+test_df['text'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['text'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['text'].values.tolist())

cv_scores = []
pred_full_test = 0
pred_train = np.zeros([train_df.shape[0], 3])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)

for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index, :] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))

print(f'Mean cv score : {np.mean(cv_scores)}')
pred_full_test = pred_full_test / 5.

# add the predictions as new fetures #
train_df['nb_cvec_char_eap'] = pred_train[:,0]
train_df['nb_cvec_char_hpl'] = pred_train[:,1]
train_df['nb_cvec_char_mws'] = pred_train[:,2]
test_df['nb_cvec_char_eap'] = pred_full_test[:,0]
test_df['nb_cvec_char_hpl'] = pred_full_test[:,1]
test_df['nb_cvec_char_mws'] = pred_full_test[:,2]

교차 검증 수치가 3.75로 아주 높네요. 하지만 이건 단어 수준의 feature들 대신에 다른 정보를 추가할 수 있으므로 마지막 모델에서 사용하겠습니다.

In [ ]:
#### Fit transform the tfidf vectorizer ###
tfidf_vec = TfidfVectorizer(ngram_range=(1,5), analyzer='char')
full_tfidf = tfidf_vec.fit_transform(train_df['text'].values.tolist()+test_df['text'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['text'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['text'].values.tolist())

cv_scores = []
pred_full_test = 0
pred_train = np.zeros([train_df.shape[0], 3])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)

for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))

print(f'Mean cv score : {np.mean(cv_scores)}')
pred_full_test = pred_full_test / 5.

# add the predictions as new fetures #
train_df['nb_tfidf_char_eap'] = pred_train[:,0]
train_df['nb_tfidf_char_hpl'] = pred_train[:,1]
train_df['nb_tfidf_char_mws'] = pred_train[:,2]
test_df['nb_tfidf_char_eap'] = pred_full_test[:,0]
test_df['nb_tfidf_char_hpl'] = pred_full_test[:,1]
test_df['nb_tfidf_char_mws'] = pred_full_test[:,2]

**Character TFIDF를 사용한 SVD:**

우리는 또한 tfidf feature들을 활용한 svd feature들을 만들 수 있고 모델링에 사용할 수 있습니다.

In [ ]:
n_comp = 20
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(full_tfidf)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
    
train_svd.columns = ['svd_char_'+str(i) for i in range(n_comp)]
test_svd.columns = ['svd_char_'+str(i) for i in range(n_comp)]
train_df = pd.concat([train_df, train_svd], axis=1)
test_df = pd.concat([test_df, test_svd], axis=1)

del full_tfidf, train_tfidf, test_tfidf, train_svd, test_svd

**XGBoost 모델:**

이 새로운 feature들 사용해서 우리는 xgboost를 다시 돌리고 결과를 평가하겠습니다.

In [ ]:
cols_to_drop = ['id','text']
train_X = train_df.drop(cols_to_drop+['author'], axis=1)
test_X = test_df.drop(cols_to_drop, axis=1)

kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
cv_scores = []
pred_full_test = 0
pred_train = np.zeros([train_df.shape[0], 3])
for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_X.loc[dev_index], train_X.loc[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runXGB(dev_X, dev_y, val_X, val_y, test_X, seed_val=0, colsample=0.7)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    break
print("cv scores : ", cv_scores)

out_df = pd.DataFrame(pred_full_test)
out_df.columns = ['EAP', 'HPL', 'MWS']
out_df.insert(0, 'id', test_id)
out_df.to_csv("sub_fe.csv", index=False)

**검증 데이터 점수는 0.3044가 나왔고 LB 점수는 0.32xx가 나왔습니다.** 모든 fold에 대해서 돌린다면 더 좋은 점수를 얻을 것입니다. 이제 중요한 변수를 다시 확인해봅시다.

In [ ]:
### Plot the important variables ###
fig, ax = plt.subplots(figsize=(12, 12))
xgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
plt.show()

기대했듯이 나이브 베이즈 feature가 top feature입니다. 이제 혼동 행렬을 통해 오분류 오차를 봅시다.

In [ ]:
cnf_matrix = confusion_matrix(val_y, np.argmax(pred_val_y, axis=1))
np.set_printoptions(precision=2)

# Plot non-normalized confusio matrix
plt.figure(figsize=(8,8))
plot_confusion_matrix(cnf_matrix, classes=['EAP', 'HPL', 'MWS'],
                     title='Confusion matrix of XGB, without normalization')
plt.show()

EAP와 MWS가 다른 것보다 더 많이 오분류된 것으로 보입니다. 우리는 잠재적으로 이런 쌍에 대해서 예측을 향상하는 feature를 만들 수 있었습니다.

**이 FE 노트북의 다음 과정:**

- feautre에 기반한 워드 임베딩을 사용하는 것
- 가능한 다른 meta feture가 있는 경우
- 문장의 감정

**더 향상하기 위한 아이디어**

- tfidf와 count vectorizer의 파라미터 튜닝
- XGB 모델과 나이브 베이즈 파라미터 튜닝
- 다른 모델을 앙상블하거나 스태킹하기